# モジュールをインポート

In [970]:
import random
import time

# 計算用の関数

### ネイピア数
$$e = \lim_{{x \to \infty}} \left(1 + \frac{1}{x}\right)^x$$

In [971]:
def napiers_logarithm(x):
    """
    ネイピア数を求める関数

    Args:
        x (float): 入力
    
    Returns:
        float: 出力
    """
    return (1 + 1 / x) ** x
napier_number = napiers_logarithm(100000000)  # e

### 自然対数の近似値

In [972]:
def ln(x, max_iter=20, tol=1e-12):
    """
    自然対数を求める関数

    Args:
        x (float): 入力
        max_iter (int): 最大反復回数
        tol (float): 許容誤差
    
    Returns:
        float: 出力
    """
    if x <= 0: raise ValueError("x must be positive")
    k = 0
    while x > 2:
        x /= 2
        k += 1
    while x < 0.5:
        x *= 2
        k -= 1
    y = x - 1  # ln(1) = 0 付近の値から開始
    for _ in range(max_iter):
        prev_y = y
        y -= (2.718281828459045**y - x) / (2.718281828459045**y)  # f(y) / f'(y)
        if abs(y - prev_y) < tol:
            break
    return y + k * 0.6931471805599453  # ln(2) ≈ 0.693147

### 平方根を計算

In [973]:
def sqrt(x):
    """
    平方根を求める関数

    Args:
        x (float): 入力
    
    Returns:
        float: 出力
    """
    tolerance = 1e-10  # 許容誤差
    estimate = x / 2.0  # 初期推定値
    while True:
        new_estimate = (estimate + x / estimate) / 2  # ニュートン法による更新
        if abs(new_estimate - estimate) < tolerance:  # 収束したら終了
            return new_estimate
        estimate = new_estimate  # 推定値を更新

# 活性化関数およびその微分関数

### シグモイド関数
$$Sigmoid(x) = \frac{1}{1 + e^{-x}}$$
### シグモイド関数の微分
$$Sigmoid'(x) = Sigmoid(x) \cdot (1 - Sigmoid(x))$$

In [974]:
def sigmoid(x, derivative=False):
    """
    Sigmoid 関数およびその微分

    Args:
        x (float): 入力
        derivative (bool): 微分を計算するかどうか
    
    Returns:
        float: 出力
    """
    if derivative:
        return sigmoid_derivative(x)
    return 1 / (1 + napier_number ** -x)

def sigmoid_derivative(x):
    """
    Sigmoid 関数の微分

    Args:
        x (float): 入力
    
    Returns:
        float: 出力
    """
    return sigmoid(x) * (1 - sigmoid(x))

### ReLU関数
$$ReLU(x) = \max(0, x)$$
### ReLU関数の微分
$$ReLU'(x) = \begin{cases} 
1 & (x > 0) \\
0 & (x ≤ 0)
\end{cases}$$

In [975]:
def relu(x, derivative=False):
    """
    ReLU 関数およびその微分

    Args:
        x (float): 入力
        derivative (bool): 微分を計算するかどうか
    
    Returns:
        float: 出力
    """
    if derivative:
        return relu_derivative(x)
    return max(0, x)

def relu_derivative(x):
    """
    ReLU 関数の微分

    Args:
        x (float): 入力
    
    Returns:
        float: 出力
    """
    return 1 if x > 0 else 0

### Leaky ReLU関数
$$LeakyReLU(x) = \begin{cases} 
x & (x > 0) \\ 
\alpha x & (x ≤ 0)
\end{cases}$$

### Leaky ReLU関数の微分
$$LeakyReLU'(x) = \begin{cases} 
1 & (x > 0) \\ 
\alpha & (x ≤ 0)
\end{cases}$$

In [976]:
def leaky_relu(x, alpha=0.01, derivative=False):
    """
    Leaky ReLU 関数およびその微分

    Args:
        x (float): 入力
        alpha (float): ハイパーパラメータ
        derivative (bool): 微分を計算するかどうか
    
    Returns:
        float: 出力
    """
    if derivative:
        return leaky_relu_derivative(x, alpha)
    return x if x > 0 else alpha * x

def leaky_relu_derivative(x, alpha=0.01):
    """
    Leaky ReLU 関数の微分

    Args:
        x (float): 入力
        alpha (float): ハイパーパラメータ
    
    Returns:
        float: 出力
    """
    return 1 if x > 0 else alpha

### 恒等関数
$$Identity(x) = x$$
### 恒等関数の微分
$$Identity'(x) = 1$$

In [977]:
def identity(x, derivative=False):
    """
    恒等関数およびその微分

    Args:
        x (float): 入力
        derivative (bool): 微分を計算するかどうか
    
    Returns:
        float: 出力
    """
    if derivative:
        return identity_derivative(x)
    return x

def identity_derivative(x):
    """
    恒等関数の微分

    Args:
        x (float): 入力(未使用)
    
    Returns:
        int: 出力
    """
    return 1

# 損失関数

### クロスエントロピー損失
$$ L = -\sum_{i=1}^{N} y_i \cdot \ln(\hat{y}_i + \epsilon)$$

In [978]:
def cross_entropy_loss(y_true, y_pred):
    """
    交差エントロピー損失関数

    Args:
        y_true (list): 正解ラベル
        y_pred (list): 予測ラベル
    
    Returns:
        float: 出力
    """
    if len(y_true) != len(y_pred): raise ValueError("Input lists must have the same length.")
    return -sum([t * ln(p + 1e-9) for t, p in zip(y_true, y_pred)])

### 平均二乗誤差
$$L = \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2$$

In [979]:
def mean_squared_error(y_true, y_pred):
    """
    平均二乗誤差を求める関数

    Args:
        y_true (list): 正解ラベル
        y_pred (list): 予測ラベル
    
    Returns:
        float: 出力
    """
    if len(y_true) != len(y_pred): raise ValueError("Input lists must have the same length.")
    return sum([(t - p) ** 2 for t, p in zip(y_true, y_pred)]) / len(y_true)

### 平均絶対誤差
$$L = \frac{1}{N} \sum_{i=1}^{N} |y_i - \hat{y}_i|$$

In [980]:
def mean_absolute_error(y_true, y_pred):
    """
    平均絶対誤差を求める関数

    Args:
        y_true (list): 正解ラベル
        y_pred (list): 予測ラベル
    
    Returns:
        float: 出力
    """
    if len(y_true) != len(y_pred): raise ValueError("Input lists must have the same length.")
    return sum([abs(t - p) for t, p in zip(y_true, y_pred)]) / len(y_true)

### バイナリ交差エントロピー
$$ L = -\frac{1}{N} \sum_{i=1}^{N} \left[ y_i \cdot \ln(\hat{y}_i + \epsilon) + (1 - y_i) \cdot \ln(1 - \hat{y}_i + \epsilon) \right] $$

In [981]:
def binary_cross_entropy_loss(y_true, y_pred):
    """
    バイナリ交差エントロピー損失関数

    Args:
        y_true (list): 正解ラベル
        y_pred (list): 予測ラベル
    
    Returns:
        float: 出力
    """
    if len(y_true) != len(y_pred): raise ValueError("Input lists must have the same length.")
    epsilon = 1e-9  # 0で割るのを防ぐための小さな値
    return -sum([t * ln(p + epsilon) + (1 - t) * ln(1 - p + epsilon) for t, p in zip(y_true, y_pred)]) / len(y_true)

### カテゴリカル交差エントロピー
$$ L = -\sum_{i=1}^{N} \sum_{j=1}^{C} y_{ij} \cdot \ln(\hat{y}_{ij} + \epsilon) $$

In [982]:
def categorical_cross_entropy_loss(y_true, y_pred):
    """
    カテゴリカル交差エントロピー損失関数

    Args:
        y_true (list): 正解ラベル
        y_pred (list): 予測ラベル
    
    Returns:
        float: 出力
    """
    if len(y_true) != len(y_pred): raise ValueError("Input lists must have the same length.")
    epsilon = 1e-9  # 0で割るのを防ぐための小さな値
    return -sum([t * ln(p + epsilon) for t, p in zip(y_true, y_pred)]) / len(y_true)

# ニューラネットワーク

### ニューラルネットワークを初期化

In [983]:
def initialize_weights(layer_sizes):  # 重みとバイアスの初期化
    """
    重みとバイアスを初期化する関数

    Args:
        layer_sizes (list): 各層のユニット数
    
    Returns:
        tuple: 重みとバイアス
    """
    weights, biases = [], []
    for i in range(len(layer_sizes) - 1):
        limit = sqrt(6 / (layer_sizes[i] + layer_sizes[i+1]))  # 重みの初期化に使う乱数の範囲
        weights.append([[random.uniform(-limit, limit) for _ in range(layer_sizes[i])] for _ in range(layer_sizes[i+1])])  # 重みは -limit から limit の間の乱数で初期化
        biases.append([0 for _ in range(layer_sizes[i+1])])  # バイアスは0で初期化
    return weights, biases

### 順伝播処理

In [984]:
def forward_propagation(inputs, weights, biases, hidden_activation, output_activation):  # 順伝播処理
    """
    順伝播処理を行う関数

    Args:
        inputs (list): 入力
        weights (list): 重み
        biases (list): バイアス
    
    Returns:
        list: 出力
    """
    activations = [inputs]
    for W, b in zip(weights, biases):
        z = [
            sum([activations[-1][i] * W[j][i] for i in range(len(activations[-1]))]) + b[j]
            for j in range(len(b))
        ]
        if W != weights[-1]:
            activations.append([hidden_activation(z_i, derivative=False) for z_i in z])
        else:
            activations.append([output_activation(z_i, derivative=False) for z_i in z])
    return activations

### 逆伝播処理

In [985]:
def backward_propagation(activations, y_true, weights, biases, learning_rate, hidden_activation, output_activation):  # 逆伝播処理
    """
    逆伝播処理を行う関数

    Args:
        activations (list): 出力
        y_true (list): 正解ラベル
        weights (list): 重み
        biases (list): バイアス
        learning_rate (float): 学習率
    
    Returns:
        tuple: 重みとバイアス
    """
    output_layer = activations[-1]
    errors = [
        (output_layer[i] - y_true[i]) * output_activation(output_layer[i], derivative=True)
        for i in range(len(y_true))
    ]
    deltas = [errors]
    # 隠れ層の誤差を計算
    for l in range(len(weights)-1, 0, -1):
        hidden_errors = [
            sum([deltas[0][k] * weights[l][k][j] for k in range(len(deltas[0]))]) * hidden_activation(activations[l][j], derivative=True)
            for j in range(len(activations[l]))
        ]
        deltas.insert(0, hidden_errors)
    # 重みとバイアスを更新
    for l in range(len(weights)):
        for i in range(len(weights[l])):
            for j in range(len(weights[l][i])):
                weights[l][i][j] -= learning_rate * deltas[l][i] * activations[l][j]
            biases[l][i] -= learning_rate * deltas[l][i]

    return weights, biases

### 学習

In [986]:
def train(X, y, layer_sizes, epochs, learning_rate, hidden_activation=relu, output_activation=sigmoid, loss=cross_entropy_loss):  # 学習
    """
    ニューラルネットワークを学習する関数

    Args:
        X (list): 入力
        y (list): 正解ラベル
        layer_sizes (list): 各層のユニット数
        epochs (int): エポック数
        learning_rate (float): 学習率
    
    Returns:
        tuple: 重みとバイアス
    """
    weights, biases = initialize_weights(layer_sizes)
    start = time.time()
    for epoch in range(epochs):
        total_loss = 0
        for i in range(len(X)):
            activations = forward_propagation(X[i], weights, biases, hidden_activation, output_activation)
            total_loss += loss(y[i], activations[-1])
            weights, biases = backward_propagation(activations, y[i], weights, biases, learning_rate, hidden_activation, output_activation)
        m = epoch // (epochs // 20) + 1
        print(f"\rEpoch {epoch+1}/{epochs}, Loss: {total_loss/len(X):.10f}, [{'+'*m}{' '*(20-m)}]{' '*5}",end="")
    print(f"Training time: {time.time()-start:.2f} seconds")
    return weights, biases

### 予測

In [987]:
def predict(X, weights, biases, hidden_activation=relu, output_activation=sigmoid):  # 予測
    """
    予測を行う関数

    Args:
        X (list): 入力
        weights (list): 重み
        biases (list): バイアス
    
    Returns:
        list: 出力
    """
    outputs = []
    for i in range(len(X)):  # Prediction
        outputs.append(forward_propagation(X[i], weights, biases, hidden_activation, output_activation)[-1])
    return outputs

### 精度計算

In [988]:
def accuracy(X, y, predict, loss=cross_entropy_loss):  # 予測精度の計算
    """
    予測精度を計算する関数

    Args:
        X (list): 入力
        y (list): 正解ラベル
        predict (list): 予測値
    
    Returns:
        float: 予測精度
    """
    total_loss = 0
    for i in range(len(predict)):  # Prediction
        print(f"入力: {X[i]}, 正解: {y[i]}, 予測値: {predict[i]}")
        total_loss += loss(y[i], predict[i])
    print(f"Loss: {total_loss / len(predict):.10f}")
    return total_loss / len(predict)

# データ処理

### 正規化

In [989]:
def normalize(data, denomalize=False, min_val=None, max_val=None):
    """
    データを正規化する関数

    Args:
        data (list): 入力データ
        denomalize (bool): 逆正規化を行うかどうか
        min_val (float): 最小値
        max_val (float): 最大値
    
    Returns:
        tuple: 正規化されたデータ、最小値、最大値もしくは逆正規化されたデータ
    """
    if denomalize:
        return [[x * (max_val - min_val) + min_val for x in sublist] for sublist in data]
    min_val = min(min(sublist) for sublist in data)
    max_val = max(max(sublist) for sublist in data)
    nomalized_data = [[(x - min_val) / (max_val - min_val) for x in sublist] for sublist in data]
    return nomalized_data, min_val, max_val

### 標準化

In [990]:
def standardize(data, unstandardize=False, mean=None, std_dev=None):
    """
    データを標準化する関数

    Args:
        data (list): 入力データ
        unstandardize (bool): 逆標準化を行うかどうか
        mean (float): 平均
        std_dev (float): 標準偏差
    
    Returns:
        tuple: 標準化されたデータ、平均、標準偏差もしくは逆標準化されたデータ
    """
    if unstandardize:
        return [[x * std_dev + mean for x in sublist] for sublist in data]
    mean = sum(sum(sublist) for sublist in data) / (len(data) * len(data[0]))
    std_dev = (sum((x - mean) ** 2 for sublist in data for x in sublist) / (len(data) * len(data[0]))) ** 0.5
    standardized_data = [[(x - mean) / std_dev for x in sublist] for sublist in data]
    return standardized_data, mean, std_dev

### ラベルエンコーディング

In [991]:
def label_encoding(labels, decoding=False, label_to_index=None):
    """
    ラベルエンコーディングを行う関数

    Args:
        labels (list): カテゴリカルデータのリスト(デコードを行う際はエンコードした元文字列(現数値)のみを指定可能)
        decoding (bool): デコードを行うかどうか(デコードを行う際は label_to_index を指定)
        label_to_index (dict): ラベルとインデックスのマッピング
    
    Returns:
        tuple: エンコードされたラベル、ラベルとインデックスのマッピング
    """
    if decoding:
        return [[list(label_to_index.keys())[list(label_to_index.values()).index(label)] for label in sublist] for sublist in labels]
    str_labels = [label for label in sum(labels, []) if type(label) == str]
    label_to_index = {label: idx for idx, label in enumerate(sorted(set(str_labels)))}
    for i in range(len(labels)):
        for j in range(len(labels[i])):
            if labels[i][j] in label_to_index and type(labels[i][j]) == str:
                labels[i][j] = label_to_index[labels[i][j]]
    return labels, label_to_index

### ワンホットエンコーディング

In [992]:
def one_hot_encoding(labels, decoding=False, label_to_index=None):
    """
    ワンホットエンコーディングを行う関数
    
    Args:
        labels (list): カテゴリカルデータのリスト(デコードを行う際はエンコードした元文字列(現数値)のみを指定可能)
        decoding (bool): デコードを行うかどうか(デコードを行う際は label_to_index を指定)
        label_to_index (dict): ラベルとインデックスのマッピング
    
    Returns:
        tuple: エンコードされたラベル、ラベルとインデックスのマッピング
    """
    if decoding:
        return[[k] for sublist in labels for i, x in enumerate(sublist) for k in label_to_index if x == 1 and i == label_to_index[k]]
    str_labels = [label for label in sum(labels, []) if type(label) == str]
    label_to_index = {label: idx for idx, label in enumerate(sorted(set(str_labels)))}
    zeros = [0] * len(label_to_index)
    for i in range(len(labels)):
        for j in range(len(labels[i])):
            if labels[i][j] in label_to_index and type(labels[i][j]) == str:
                labels[i][j] = zeros[:label_to_index[labels[i][j]]] + [1] + zeros[label_to_index[labels[i][j]]+1:]
    return [[x for y in sublist for x in (y if isinstance(y, list) else [y])] for sublist in labels], label_to_index

### データセット分割

In [993]:
def split_dataset(X, y, train_size=0.8):  # データセットを学習用とテスト用に分割
    """
    データセットを学習用とテスト用に分割する関数

    Args:
        X (list): 入力
        y (list): 正解ラベル
        train_size (float): 学習データの割合
    
    Returns:
        tuple: 学習用データとテスト用データ
    """
    n = len(X)
    indices = list(range(n))
    random.shuffle(indices)
    X_train, y_train = [X[i] for i in indices[:int(n*train_size)]], [y[i] for i in indices[:int(n*train_size)]]
    X_test, y_test = [X[i] for i in indices[int(n*train_size):]], [y[i] for i in indices[int(n*train_size):]]
    return X_train, y_train, X_test, y_test

### メインコード

In [994]:
# データセット
# X = [[0, 0], [0, 1], [1, 0], [1, 1]]  # 入力
# y = [[1], [0], [0], [1]]  # 出力

# X = [[random.random(), random.random()] for _ in range(100)]
# y = [[1] if x[0] + x[1] > 1 else [0] for x in X]

# X = [[random.random(), random.random()] for _ in range(100)]
# y = [[x[0] + x[1]] for x in X]

# X = [[0, "A"], [1, "A"], [2, "B"], [3, "A"], [4, "B"], [5, "A"], [6, "B"], [7, "B"], [8, "A"], [9, "B"]]
# y = [[1], [1], [0], [1], [0], [1], [0], [0], [1], [0]]

X = [["A", "A"], ["A", "A"], ["B", "B"], ["A", "A"], ["B", "B"], ["A", "A"], ["B", "B"], ["B", "B"], ["A", "A"], ["B", "B"]]
y = [[1], [1], [0], [1], [0], [1], [0], [0], [1], [0]]

# X, min_val, max_val = normalize(X)
X, label_to_index = label_encoding(X)

X_train, y_train, X_test, y_test = split_dataset(X, y, train_size=0.8)

epochs = 1000  # エポック数
learning_rate = 0.01  # 学習率
layer_sizes = [len(X[0]), 8, 16, 8, len(y[0])]  # 各層のユニット数

hidden_activation = leaky_relu
output_activation = identity
loss = mean_absolute_error

# X_train: 入力, y_train: 出力, layer_sizes: 各層のユニット数, epochs: エポック数, learning_rate: 学習率, hidden_activation: 隠れ層の活性化関数, output_activation: 出力層の活性化関数, loss: 損失関数
weights, biases = train(X_train, y_train, layer_sizes, epochs, learning_rate, hidden_activation=hidden_activation, output_activation=output_activation, loss=loss)
# X_test: テスト入力, weights: 重み, biases: バイアス, hidden_activation: 隠れ層の活性化関数, output_activation: 出力層の活性化関数
predict_y = predict(X_test, weights, biases, hidden_activation=hidden_activation, output_activation=output_activation)

# X_test: テスト入力, y_test: テスト正解ラベル, predict_y: 予測値, loss: 損失関数
accuracy_num = accuracy(X_test, y_test, predict_y, loss=loss)

Epoch 1000/1000, Loss: 0.0000000000, [++++++++++++++++++++]     Training time: 0.91 seconds
入力: [1, 1], 正解: [0], 予測値: [5.551115123125783e-16]
入力: [1, 1], 正解: [0], 予測値: [5.551115123125783e-16]
Loss: 0.0000000000


# 活性化関数

### Sigmoid関数

| **用途**                     | **隠れ層での使用**                                | **出力層での使用**                                       | **備考**                                                                 |
|------------------------------|------------------------------------------------|-------------------------------------------------------|--------------------------------------------------------------------------|
| **バイナリ分類**              | ×                                              | ◎ 確率を出力するために使用                              | 出力が0～1の範囲で、クラス1の確率として解釈可能。                          |
| **多クラス分類（マルチラベル）** | △ 隠れ層で補助的に使用することも可能              | ◎ 各クラスの独立した確率出力に使用                      | 各クラスごとに個別の確率を算出する場合に適用（例: 複数ラベル分類）。         |
| **多クラス分類（マルチクラス）** | ×                                              | × Softmax関数が主流                                    | 複数のクラスにまたがる分類タスクではSoftmaxが適している。                   |
| **回帰問題**                  | △ 隠れ層で使用する場合もある                      | △ 出力を正規化する場合のみ使用可能                      | 出力範囲が0～1に制約される必要がある場合に限る。                           |
| **時系列予測（ゲート構造）**   | ◎ RNNやLSTMのゲートで使用される                  | ×                                              | 入力ゲートや忘却ゲートなど、情報の取捨選択に有効。                         |
| **確率的タスク**               | △ 特定の計算の一部として利用可能                  | ◎ 出力を確率として解釈する場合に使用                    | ベイズ的アプローチや確率的推論に適用可能。                                 |
| **制約付き出力**               | △ 必要に応じて正規化に使用                        | ◎ 範囲を0～1に制限する場合に使用                        | 例: 需要予測、正規化された出力を要する経済学や物理学モデル。               |
| **生物学的モデリング**         | ◎ ニューロン発火モデルなどで利用                  | △ 特定のタスクに限定                                   | 神経細胞の発火特性などS字型の挙動を模倣するタスクに有効。                   |

### ReLU関数

| **用途**                     | **隠れ層での使用**                                | **出力層での使用**                                       | **備考**                                                                 |
|------------------------------|------------------------------------------------|-------------------------------------------------------|--------------------------------------------------------------------------|
| **バイナリ分類**              | ◎                                              | ×                                                     | 出力層でのSigmoidの方が適しており、ReLUは主に隠れ層で使用される。           |
| **多クラス分類（マルチラベル）** | ◎ 隠れ層での使用に適している                       | ×                                                     | 各クラスの確率出力にはSoftmaxやSigmoidが適しているため、ReLUは隠れ層で使用。 |
| **多クラス分類（マルチクラス）** | ◎ 隠れ層での使用に最適                            | × Softmaxが主流                                        | 隠れ層での非線形性に優れたReLUが広く使われ、出力層にはSoftmaxを使用。         |
| **回帰問題**                  | ◎ 隠れ層での使用に適している                       | △ 恒等関数（線形関数）が一般的                          | 出力層ではReLUを使うことは少ないが、正の値が重要な場合に使用されることがある。  |
| **時系列予測（ゲート構造）**   | ◎ RNNやLSTMの隠れ層で広く使用                    | ×                                                     | 隠れ層での非線形性が強化され、勾配消失問題に対処可能。                     |
| **確率的タスク**               | ◎ 隠れ層で使用されることが多い                      | ×                                                     | 出力層では確率を求めるためにSigmoidやSoftmaxが使われる。                   |
| **制約付き出力**               | ◎ 出力が正の値のみ必要な場合に使用                  | × 出力範囲が制約される場合には他の関数を使用               | 正の値に限定される場合の隠れ層で非常に効果的。                             |
| **生物学的モデリング**         | ◎ ニューロンの発火モデルで利用                      | ×                                                     | 活性化関数としては、生物学的ニューロンの特性に合う場合もあるが、一般にはReLUは隠れ層に使用。 |

### Leaky ReLU関数

| **用途**                     | **隠れ層での使用**                                | **出力層での使用**                                       | **備考**                                                                 |
|------------------------------|------------------------------------------------|-------------------------------------------------------|--------------------------------------------------------------------------|
| **バイナリ分類**              | ◎ 隠れ層で使用されることが多い                      | ×                                                     | 出力層でのSigmoidの方が適しており、Leaky ReLUは隠れ層で使用されることが一般的。 |
| **多クラス分類（マルチラベル）** | ◎ 隠れ層で使用されることが多い                      | ×                                                     | SigmoidやSoftmaxが出力層で使用されるため、Leaky ReLUは主に隠れ層で利用される。 |
| **多クラス分類（マルチクラス）** | ◎ 隠れ層での使用に最適                            | × Softmaxが主流                                        | 出力層ではSoftmax、隠れ層ではLeaky ReLUが広く使われる。                   |
| **回帰問題**                  | ◎ 隠れ層で使用されることが多い                      | △ 恒等関数（線形関数）が一般的                          | 出力層ではReLUや恒等関数を使うことが一般的だが、Leaky ReLUは隠れ層で使われる。  |
| **時系列予測（ゲート構造）**   | ◎ RNNやLSTMの隠れ層で使用されることがある           | ×                                                     | 勾配消失問題に強いため、LSTMやGRUのようなRNNアーキテクチャで有効。        |
| **確率的タスク**               | ◎ 隠れ層で使用されることが多い                      | ×                                                     | 出力層で確率を求めるためにはSigmoidやSoftmaxが使用される。                   |
| **制約付き出力**               | ◎ 出力が正の値のみ必要な場合に使用                  | × 出力範囲が制約される場合には他の関数を使用               | 正の値が重要な場合に、隠れ層でReLUの代わりに使用されることが多い。            |
| **生物学的モデリング**         | ◎ ニューロンの発火モデルで利用                      | ×                                                     | ReLUよりも小さな負の出力を許容するため、発火モデルにおいても有効。            |

### Identity関数（恒等関数）

| **用途**                     | **隠れ層での使用**                                | **出力層での使用**                                       | **備考**                                                                 |
|------------------------------|------------------------------------------------|-------------------------------------------------------|--------------------------------------------------------------------------|
| **バイナリ分類**              | ×                                              | ×                                                     | 恒等関数はバイナリ分類では通常使用されません。出力層にはSigmoidが適している。 |
| **多クラス分類（マルチラベル）** | ×                                              | ×                                                     | 出力層にはSoftmaxやSigmoidが使用されるため、恒等関数は使用されません。         |
| **多クラス分類（マルチクラス）** | ×                                              | ×                                                     | 出力層にはSoftmaxが主流であり、恒等関数は使用されません。                     |
| **回帰問題**                  | ◎ 隠れ層での使用が稀にあるが、通常は他の関数（ReLUなど）を使用 | ◎ 回帰問題では出力層で最も一般的に使用される               | 恒等関数は回帰問題の出力層で広く使用される。値の範囲に制約がないため、連続的な数値予測に最適。 |
| **時系列予測（ゲート構造）**   | ×                                              | ×                                                     | 時系列予測においては、ReLUやLSTMなど他の関数が使用される。                     |
| **確率的タスク**               | ×                                              | ×                                                     | 確率出力が求められるタスクにはSigmoidやSoftmaxが使用されるため、恒等関数は使用されません。 |
| **制約付き出力**               | ×                                              | △ 出力が特定の範囲に収まる必要がある場合に使用されることがある  | 恒等関数は制約なしに連続的な出力を生成するため、特定の範囲制約がない場合に使用されます。 |
| **生物学的モデリング**         | ×                                              | ×                                                     | 生物学的なモデルや発火モデルなどでは使用されません。                          |


# 損失関数の説明

### Cross Entropy Loss（クロスエントロピー誤差）

| **用途**                     | **定義**                                           | **適用される問題**                                   | **活性化関数**                           | **備考**                                                                 |
|------------------------------|--------------------------------------------------|----------------------------------------------------|-----------------------------------------|--------------------------------------------------------------------------|
| **バイナリ分類**              | 2クラス（0または1）に対する予測と実際のラベルの間の誤差 | バイナリ分類問題                                    | 出力層: Sigmoid                        | 出力が0～1の確率で、クラス1の確率を予測する。                             |
| **多クラス分類（マルチクラス）** | 複数クラスの中から正解クラスを予測する誤差             | マルチクラス分類問題                                | 出力層: Softmax                        | ソフトマックス関数を用いて、各クラスに対する確率を出力し、最大の確率を持つクラスが選ばれる。 |
| **多クラス分類（マルチラベル）** | 複数のラベルを同時に予測する誤差                     | 各ラベルが独立した多クラス分類問題（複数ラベル）    | 出力層: Sigmoid                        | 各クラスごとに独立した確率が出力され、複数のクラスを同時に予測する場合に使用される。 |
| **回帰問題**                  | 使用されない                                      | 回帰問題                                            | 恒等関数（Identity）または他の回帰関数 | 回帰問題ではCross Entropy Lossは一般的に使用されません。                |
| **確率的タスク**               | 出力確率を求めるタスクで、予測と実際の確率の誤差を計算 | 確率的な予測が必要なタスク（例: パラメータ推定）    | 出力層: SigmoidまたはSoftmax           | 確率的なタスクや予測問題で誤差を最小化するために使用される。               |
| **ニューラルネットワークのトレーニング** | モデルが予測する確率と実際のラベルとの誤差を計算     | ネットワークの学習中の誤差計算                      | 出力層: SigmoidまたはSoftmax           | クロスエントロピー誤差は多くの分類問題の損失関数として標準的に使用される。 |

### Mean Squared Error (MSE)
| **用途**               | **定義**                                          | **適用される問題**                          | **活性化関数**            | **備考**                                                                 |
|------------------------|-------------------------------------------------|-------------------------------------------|--------------------------|--------------------------------------------------------------------------|
| **回帰問題**            | 実際の値と予測値の二乗誤差の平均                    | 回帰問題（連続的な数値予測）                | 恒等関数（Identity）       | MSEは回帰問題でよく使用され、誤差が大きい予測に対してペナルティを強く与えます。   |
| **ノイズ除去**          | ノイズ除去タスクで、実際の出力と予測出力との差の二乗平均  | 信号処理や画像のノイズ除去など               | 恒等関数（Identity）       | 特にノイズが含まれるデータの学習において、予測誤差を最小化するために使用されます。 |

### Mean Absolute Error (MAE)
| **用途**               | **定義**                                          | **適用される問題**                          | **活性化関数**            | **備考**                                                                 |
|------------------------|-------------------------------------------------|-------------------------------------------|--------------------------|--------------------------------------------------------------------------|
| **回帰問題**            | 実際の値と予測値の絶対誤差の平均                    | 回帰問題（連続的な数値予測）                | 恒等関数（Identity）       | MAEは外れ値に敏感でなく、誤差が小さい場合に使いやすい損失関数です。        |
| **ロバスト回帰**        | 外れ値に対して頑健な回帰問題                     | 外れ値が含まれる回帰問題                    | 恒等関数（Identity）       | MSEが外れ値に強く反応するのに対し、MAEは外れ値に対してロバストです。      |

### Binary Cross-Entropy Loss
| **用途**               | **定義**                                          | **適用される問題**                          | **活性化関数**            | **備考**                                                                 |
|------------------------|-------------------------------------------------|-------------------------------------------|--------------------------|--------------------------------------------------------------------------|
| **バイナリ分類**        | 2クラス（0または1）の予測と実際のラベルの間の誤差 | バイナリ分類問題（例: スパム分類）            | 出力層: Sigmoid           | 出力が0〜1の確率として解釈され、最も確信度の高いクラスを選択する。         |
| **確率的推定**          | 予測確率と実際のラベルの間の誤差を計算           | 確率的な分類タスク（例: 二項分布のパラメータ推定）| 出力層: Sigmoid           | 確率的な推定やパラメータ推定など、確率を扱うタスクに使用されます。          |

### Categorical Cross-Entropy Loss
| **用途**               | **定義**                                          | **適用される問題**                          | **活性化関数**            | **備考**                                                                 |
|------------------------|-------------------------------------------------|-------------------------------------------|--------------------------|--------------------------------------------------------------------------|
| **多クラス分類**        | 複数クラスの中から正解クラスを予測する誤差        | 多クラス分類問題（例: 手書き数字認識）        | 出力層: Softmax           | ソフトマックス関数を使用し、各クラスの確率を出力。最大確率を持つクラスが選ばれる。 |
| **確率的推定**          | 各クラスの確率と実際のラベルとの誤差を計算       | 確率的な分類タスク（例: 多項分布のパラメータ推定）| 出力層: Softmax           | 確率的な分類タスクやパラメータ推定で使用され、出力確率を最大化します。         |